In [1]:
import pysrt
from bs4 import BeautifulSoup, Tag
import gensim
import nltk.data
from gensim.models import word2vec
from nltk.corpus import stopwords
import os
from nltk.stem import WordNetLemmatizer
import logging
#nltk.download()

In [179]:
wordnet_lemmatizer = WordNetLemmatizer()

In [180]:
wordnet_lemmatizer.lemmatize('dogs')

u'dog'

In [181]:
print subs.path[4:]

1x01 - Winter is Coming.srt


In [723]:
def clean_text(text):
    text = text.replace('\n',' ')
    soup = BeautifulSoup(text)

    for tag in soup.find_all('font'):
        tag.replaceWith('')
    text = soup.get_text()
    text = text.replace('they\'re','they are')
    text = text.replace('They\'re','They are')
    text = text.replace('They\'ve','They have')
    text = text.replace('they\'ve','they have')
    text = text.replace('I\'ve','I have')
    text = text.replace('won\'t','would not')
    text = text.replace('don\'t','do not')
    text = text.replace('Don\'t','Do not')
    text = text.replace('he\'ll','he will')
    text = text.replace('It\'s','It is')
    text = text.replace('it\'s','it is')
    text = text.replace('we\'re','we are')
    text = text.replace('you\'ve','you have')
    text = text.replace('You\'ve','You have')
    text = text.replace('You\'re','You are')
    text = text.replace('you\'re','you are')
    text = text.replace('he\'s','he is')
    text = text.replace('He\'s','He is')
    text = text.replace('she\'s','she is')
    text = text.replace('She\'s','She is')
    text = text.replace('I\'m','I am')
    text = text.replace('one\'s','one is')
    text = text.replace('We\'re','We are')
    text = text.replace('we\'re','we are')
    text = text.replace('didn\'t','did not')
    text = text.replace('That\'s','That is')
    text = text.replace('that\'s','that is')
    text = text.replace('There\'s','There is')
    text = text.replace('there\'s','there is')
    text = text.replace('We\'ll','We will')
    text = text.replace('we\'ll','we will')
    text = text.replace('We\'ve','We have')
    text = text.replace('we\'ve','we have')
    text = text.replace('Where\'s','Where is')
    text = text.replace('where\'s','where is')
    text = text.replace('haven\'t','have not')
    text = text.replace('we\'d','we would')
    text = text.replace('Isn\'t','Is not')
    text = text.replace('isn\'t','is not')
    text = text.replace('you\'d','you would')
    text = text.replace('You\'d','You would')
    text = text.replace('I\'d','I would')
    text = text.replace('aren\'t','are not')
    text = text.replace('you\'ll','you will')
    text = text.replace('You\'ll','You will')
    text = text.replace('it\'ll','it will')
    text = text.replace('It\'ll','It will')
    text = text.replace('weren\'t','were not')
    text = text.replace('men','man')
    text = text.replace('lannisters','lannister')
    text = text.replace('robb\'s','robb')
    text = text.replace('wasn\'t','was not')
    text = text.replace('\'','')
    text = text.replace('!','')
    text = text.replace('-','')
    text = text.replace('==','')
    text = text.replace(',',' ')
    text = text.replace('.',' ')
    text = text.replace('?',' ')
    text = text.replace('"',' ')
    text = text.lower()
    return text

In [878]:
def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    review_text = review
    # 1. Remove HTML
    #review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    #review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    more_stops = ['would','ill','come','one','up.','up','whose','get','',' ','well','say','see','going',
                 'like','tell','want','make','know','year','go','yes','take','time','never','could','need',
                 'let','enough','many','keep','nothing','oh','look','father','think','cant','thing','still',
                  'even','heard','call','back','hear','u','ever','said','better','every','find','may','word',
                 'boy','man','lady','woman','give','must','day','done','right','good','always','little','long',
                 'seven','girl','son','brother','way','child','king','lord','mother','away','got','whats','ask',
                 'wanted','put','first','much','something','friend','sure','course','told','made','war','god',
                 'old','people','world']
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [wordnet_lemmatizer.lemmatize(w) for w in words if not w in stops] # lemmatize
        words = [w.strip() for w in words if not w in more_stops]
    #
    # 5. Return a list of words
    return(words)

In [879]:
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [880]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [991]:
files = os.listdir('s04/')
sentences = []
for f in files:
    try:
        subs = pysrt.open('s04/'+f)
    except:
        continue
    text = clean_text(subs.text)
    sentences += review_to_sentences(text,tokenizer,remove_stopwords=True)

In [992]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
        level=logging.INFO)

# Set values for various parameters
num_features = 500    # Word vector dimensionality                      
min_word_count = 2   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 30          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
print "Training model..."
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_4minwords_10context"
model.save(model_name)


Training model...


In [993]:
#model.vocab

In [994]:
model.most_similar('baratheon',topn=5)

[(u'wall', 0.9999886155128479),
 (u'night', 0.9999884366989136),
 (u'name', 0.9999882578849792),
 (u'die', 0.9999881982803345),
 (u'dead', 0.9999881386756897)]

In [995]:
#model.vocab

In [996]:
model.doesnt_match('arya robb sansa snow catelyn'.split())

'robb'

In [997]:
model.similarity('arya','hound')

0.99995842077601482